<a href="https://colab.research.google.com/github/loftusa/linear_sum_assignment/blob/main/knowledge_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic updating algorithm

$\hat{P}_{b,l}^{(i)} = (1-\epsilon)\frac{\sum_{b', b' \neq b} \sum_{l'} S_{l, l'} \hat{P}_{b', l'}^{(i-1)}}{\sum_{b', b' \neq b} \sum_{l'} S_{l, l'}} + \epsilon P_{b,l}$

Can initialize arbitrarilty, so we'll just use the prior $P_{b,l}$

Initalization value is $\frac{1}{nlabels}$

In [10]:
import numpy as np

# fork, spoon, monkey
# first bounding box has a fork
# second bounding box has a spoon
# goal: P_hat should end up having a higher value
# for the argmax of each row, and a lower value for the rest.
P = np.array([[.7, .6, .4],
              [.6, .9, .5]])

# S: fork, spoon, monkey
S = np.array([[.5, .7, .2],
              [0,  .5, .15],
              [0, 0, .5]])
S = S + S.T

array([[1.  , 0.7 , 0.2 ],
       [0.7 , 1.  , 0.15],
       [0.2 , 0.15, 1.  ]])

In [21]:
import torch

device = "cpu"
e = .5
b, l = 0, 0
nb, nl = P.shape  # num bounding boxes, num labels

numerator = 0
denominator = 0
for b_ in range(nb):
  if b_ != b:
    for l_ in range(nl):
      numerator += (S[l, l_] * P[b_, l_])
      denominator += S[l, l_]

In [35]:
numerator

1.03

In [36]:
denominator

1.4

In [31]:
S @ P.T

tensor([[0.8500, 1.0300],
        [0.3600, 0.5250],
        [0.2000, 0.2500]], dtype=torch.float64)

In [34]:
torch.mm(S, torch.transpose(P, 0, 1))

tensor([[0.8500, 1.0300],
        [0.3600, 0.5250],
        [0.2000, 0.2500]], dtype=torch.float64)

In [39]:
P_hat = torch.full(size=P.shape, fill_value=(1/nl), dtype=torch.double).to(device)
if not isinstance(P, torch.Tensor):
  P = torch.from_numpy(P).to(device)
if not isinstance(S, torch.Tensor):
  S = torch.from_numpy(S).to(device)
  
torch.sum(torch.mm(S, torch.transpose(P, 0, 1)), 1)


tensor([1.8800, 0.8850, 0.4500], dtype=torch.float64)

In [ ]:
torch.sum(torch.mm(S, P_hat))

In [ ]:
import torch

device = "gpu"
e = .5
b, l = 0, 0
num_boxes, num_classes = P.shape  # num bounding boxes, num labels
numerator = 0
denominator = 0
num_iterations = 10

p_hat_init = torch.full(size=(num_boxes, num_classes), fill_value=(1 / num_classes), dtype=torch.double).to(device)
p_hat = p_hat_init
for i in range(num_iterations):
    p_hat_temp = torch.clone(p_hat)
    for b in range(num_boxes):
        p = predictions[b]
        num = torch.sum(torch.mm(S_highest, torch.transpose(p_hat_temp[box_nearest[b]], 0, 1)), 1)
        denom = torch.sum(S_highest, dim=1) * bk
        p_hat[b] = (1 - epsilon) * torch.squeeze(torch.div(num, denom)) + epsilon * p
        p_hat[b] = torch.nan_to_num(p_hat[b])

return p_hat

In [12]:
import torch

device = "gpu"
e = .5
b, l = 0, 0
nb, nl = P.shape  # num bounding boxes, num labels
numerator = 0
denominator = 0

for b_ in range(nb):
  if b_ != b:
    for l_ in range(nl):
      numerator += (S[l, l_] * P[b_, l_])
      denominator += S[l, l_]

def find_p_hat(boxes, predictions, bk, lk, S, num_iterations, epsilon):
    """
    Compute the knowledge aware predictions, based on the object detector's output and semantic consistency.
    :param boxes: tensor of bounding boxes
    :param predictions: tensor of prediction scores
    :param bk: number of neighbouring bounding boxes to consider for p_hat
    :param lk: number of largest semantic consistent classes to consider for p_hat
    :param S: semantic consistency matrix
    :param num_iterations: number of iterations to calculate p_hat
    :param epsilon: trade-off parameter for traditional detections and knowledge aware detections
    :return p_hat: tensor of knowledge aware predictions
    """

    num_boxes = predictions.shape[0]
    num_classes = predictions.shape[1]

    if num_boxes <= 1:
        return predictions

    if num_boxes <= bk:
        bk = num_boxes - 1

    if num_classes <= lk:
        lk = num_classes

    box_centers = torch.empty(size=(num_boxes, 2), dtype=torch.double).to(device)
    box_centers[:, 0] = ((boxes[:, 2] - boxes[:, 0]) / 2) + boxes[:, 0]
    box_centers[:, 1] = ((boxes[:, 3] - boxes[:, 1]) / 2) + boxes[:, 1]

    box_nearest = torch.empty(size=(num_boxes, bk), dtype=torch.long).to(device)
    for i in range(len(boxes)):
        box_center = box_centers[i]
        distances = torch.sqrt((box_center[0] - box_centers[:, 0]) ** 2 + (box_center[1] - box_centers[:, 1]) ** 2)
        distances[i] = float('inf')
        box_nearest[i] = torch.argsort(distances)[0:bk]

    S_highest = torch.zeros(size=(num_classes, num_classes), dtype=torch.double).to(device)
    for i in range(len(S)):
        S_args = torch.argsort(S[i])[-lk:]
        S_highest[i, S_args] = S[i, S_args]

    p_hat_init = torch.full(size=(num_boxes, num_classes), fill_value=(1 / num_classes), dtype=torch.double).to(device)
    p_hat = p_hat_init
    for i in range(num_iterations):
        p_hat_temp = torch.clone(p_hat)
        for b in range(num_boxes):
            p = predictions[b]
            num = torch.sum(torch.mm(S_highest, torch.transpose(p_hat_temp[box_nearest[b]], 0, 1)), 1)
            denom = torch.sum(S_highest, dim=1) * bk
            p_hat[b] = (1 - epsilon) * torch.squeeze(torch.div(num, denom)) + epsilon * p
            p_hat[b] = torch.nan_to_num(p_hat[b])

    return p_hat

In [13]:

numerator

1.03

In [14]:
denominator

1.4